<a href="https://colab.research.google.com/github/Tanishtha-Reddy/Twitter-Sentiment-Analyzer/blob/main/Twitter_Sentiment_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
#adding the kaggle.json path
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#importing the twitter sentiment dataset
! kaggle datasets download -d kazanova/sentiment140
! unzip sentiment140.zip

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1734, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open sentiment140.zip, sentiment140.zip.zip or sentiment140.zip.ZIP.


In [ ]:
#to exctract the compressed dataset from zip file

from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

FileNotFoundError: [Errno 2] No such file or directory: '/content/sentiment140.zip'

In [ ]:
#importing the dependencies

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#downloading stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#to print the stopwords in English (they dont add any influential
#meaning to the tweet, so remove these words from the ML model)
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
#To process our dataset
#loading the data from csv file to pandas dataframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

FileNotFoundError: [Errno 2] No such file or directory: '/content/training.1600000.processed.noemoticon.csv'

In [ ]:
#checking the number of rows(total number of tweets) and columns
twitter_data.shape

(1599999, 6)

In [ ]:
#printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#to name the colums and rereading the dataset
column_names=['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding = 'ISO-8859-1')

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
#to check if any values are missing/ to count the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
#to check for the distribution of target columns( positive and negative tweets)
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
#convert the target "4" to "1"
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
#0 -->Negative tweet
#1-->Positive tweet
#Stemming is the process of reducing a word to its root word(verb?)
port_stem=PorterStemmer()#to stem and reduce the words, simplifies

In [ ]:
def stemming(content):
  #we will the text column & remove eveerything that is not an alphabet
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  #to convert uppercase to lowercase
  stemmed_content=stemmed_content.lower()
  #split all the words in a tweet and store it in a list
  stemmed_content=stemmed_content.split()
  #to do stemming, and reduce the space used, also take out stopwords and stemmed words
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #combine the result tweet after all the processing and join them back in a list
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
#to apply the above function
#new column named stemmed_content is made
#takes 50 minutes to complete this step
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()#shows even stemmed part now too

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#to seperate the data and label

X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
#split the dataset into training and test datasets

In [ ]:
#X_train-->contains all training data tweets
#X_test-->contains all testing data tweets
#Y_train-->contains all training data labels
#Y_test-->contains all testing data labels
#test_size=0.2-->20% data goes to testing, 80% goes to training
#stratify=Y-->makes a split so that the proportion of avlues in the sample
#produced will be the same as the proportion of values.
X_train, X_test, Y_train, Y_test=train_test_split(X,Y, test_size=0.2, stratify=Y,random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
#Feature Extraction
#Convert textual data to numerical data
vectorizer=TfidfVectorizer()#it will assign values to each word, either 0 or 1

X_train = vectorizer.fit_transform(X_train)#to fit the training data, assign values to them

X_test = vectorizer.transform(X_test)
#Error because they are already into numerical form :)

AttributeError: 'csr_matrix' object has no attribute 'lower'

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove special characters
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Example:
sample_tweet = "I love this movie! #awesome"
print(preprocess_text(sample_tweet))


love movie awesome


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0, solver='liblinear', max_iter=1000)


In [ ]:
from sklearn.linear_model import LogisticRegression
#Training the logistic regression model
# Logistic Regression Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)  # Train the model

# Make predictions on test data
#Y_pred = model.predict(X_test)

# Check predictions
#print(Y_pred)


LogisticRegression(max_iter=1000)

In [ ]:
#testing how accurate the model is in training data = ~80%
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train, X_train_prediction)

In [ ]:
print("Accuracy score on training data = ", training_data_accuracy)

Accuracy score on training data =  0.79871953125


In [ ]:
#testing how accurate the model is in test data = ~77%
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test, X_test_prediction)

In [ ]:
print("Accuracy score on test data = ", test_data_accuracy)

Accuracy score on test data =  0.77668125


In [ ]:
#to save the ML model
import pickle
filename = 'training_model.save'
#saving the model in pickle
pickle.dump(model, open(filename, 'wb'))#in writing mode

In [ ]:
#Using the saved model for future predictions
#loading the saved model
loaded_model = pickle.load(open('/content/training_model.save', 'rb'))#in writing mode

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction=model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("Negative tweet")

else:
  print("Positive tweet")

1
[1]
Positive tweet


In [ ]:
import numpy as np

# Count the number of positive and negative tweets
positive_count = np.sum(y_train)
negative_count = len(y_train) - positive_count

print(f"Positive tweets: {positive_count}")
print(f"Negative tweets: {negative_count}")


Positive tweets: 2
Negative tweets: 2


In [ ]:
X_new = X_test[3]
print(Y_test[3])

prediction=model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("Negative tweet")

else:
  print("Positive tweet")

0
[0]
Negative tweet


In [ ]:
#For USER INPUT
import pickle

# Load vectorizer and model
with open("/content/vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

with open("/content/model.pkl", "rb") as f:
    model = pickle.load(f)

# Take user input
user_tweet = input("Enter a tweet: ")

# Convert text input into the same numerical format
user_tweet_vectorized = vectorizer.transform([user_tweet])

# Predict sentiment
predicted_sentiment = model.predict(user_tweet_vectorized)

# Display result
if predicted_sentiment[0] == 0:
    print("Sentiment: Negative tweet")
else:
    print("Sentiment: Positive tweet")


Enter a tweet: happy
Sentiment: Positive tweet
